### Jung Suk Lee, JL5241@Columbia.edu, JL5241

### Set up

In [77]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from scipy.stats import shapiro

In [78]:
df = pd.read_csv('winequalityred.csv')

In [79]:
col_names = [i.replace("\"","").replace('\'','') for i in df.columns[0].split(';')]

In [80]:
df = pd.DataFrame(df.iloc[:,0].str.split(';').tolist(),columns = col_names)
df = df.apply(pd.to_numeric)

In [81]:
X = df.drop('quality', axis = 1)

In [82]:
y = df['quality']

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.2, 
                                                    random_state=0)

### Regression equations and functions

\begin{equation}
 y =  \sum_{j=1}^{p} X_{j}*B_{j} 
\end{equation}
\begin{equation}
RSS =  \sum_{i=1}^{N} (y_{i} - B_{o} - \sum_{j=1}^{p} x_{ij}*B_{j})^2
\end{equation}


In [84]:
def lm_predict(X, beta):
    return X.apply(lambda z: np.dot(z, beta), axis = 1)

In [85]:
def RSS(beta, X, y):
    return ((y - lm_predict(X, beta))**2).sum()

### Optimizing the Model

In [86]:
bo = np.random.normal(0,1,X_train.shape[1])

In [87]:
res = minimize(RSS, x0= np.random.normal(0,1,X_train.shape[1]), args = (X_train, y_train), tol=10**-3)

In [88]:
beta_hat = res.x

### Qualitative Results

In [89]:
beta_hat[(-beta_hat).argsort()]

array([ 4.19185798e+00,  8.74711237e-01,  3.00696782e-01,  1.31235832e-02,
        8.44059572e-03,  3.02578789e-03, -2.87979537e-03, -1.79425465e-01,
       -4.27625892e-01, -1.17657071e+00, -1.93704486e+00])

In [90]:
###largest to smallest
X.columns[(-beta_hat).argsort()]

Index(['density', 'sulphates', 'alcohol', 'residual sugar', 'fixed acidity',
       'free sulfur dioxide', 'total sulfur dioxide', 'citric acid', 'pH',
       'volatile acidity', 'chlorides'],
      dtype='object')

In [91]:
np.mean(X['total sulfur dioxide'])

46.46779237023139

In [92]:
np.mean(X['density'])

0.9967466791744833

Features that are positively correlated to good quality are density, sulphates, alcohol, residual sugar, fixed acidity, free sulfur dioxide. 

Features that are negatively correlated to good quality are citric acid, pH, volaitile acitidy and chlorides. 

Scale of the features can have a large impact to the size of the coefficients. For example, density is averaged at .9967. So if we were to increase it by just one increment that would be double the average. On the other hand, total sulfur dioxide's average is at 46.46 so increment of one is not a proportionally large change.

### RSS in Test Data

In [93]:
RSS(beta_hat, X_train, y_train)

545.5359871247364

In [94]:
RSS(beta_hat, X_test, y_test)

122.42626874968093

In [95]:
shapiro((y_test-lm_predict(X_test, beta_hat)))

(0.9787293672561646, 0.00011085698497481644)

Our model fits relatively well in test data than in the train data. Our RSS is much smaller in the test data at 122.42 as opposed to 545, which suggests that the model generalizes well. However, when we run the shaprio wilks test on the residuals, we see that we reject the null hypothesis that the residuals are normally distributed, which is a strong assumption of linear models. Therefore, there is a reason to believe that linear regression does not fit very well in this data and some of the assumptions are violated.

### Exploring Different Initial $B_o$ Values

In [101]:
res_zeros = minimize(RSS, x0= np.zeros(X_train.shape[1]), args = (X_train, y_train), tol=10**-3)

In [102]:
res_zeros.fun

545.5359871268739

In [103]:
res_zeros.x

array([ 8.44032481e-03, -1.17656945e+00, -1.79422918e-01,  1.31235183e-02,
       -1.93706309e+00,  3.02580050e-03, -2.87980093e-03,  4.19187086e+00,
       -4.27628436e-01,  8.74711492e-01,  3.00696602e-01])

In [104]:
beta_hat

array([ 8.44059572e-03, -1.17657071e+00, -1.79425465e-01,  1.31235832e-02,
       -1.93704486e+00,  3.02578789e-03, -2.87979537e-03,  4.19185798e+00,
       -4.27625892e-01,  8.74711237e-01,  3.00696782e-01])

In [109]:
res_large = minimize(RSS, x0= np.random.normal(0,1000,X_train.shape[1]), args = (X_train, y_train), tol=10**-3)

In [110]:
res_large.fun

545.5359871237804

In [111]:
res_large.x

array([ 8.44081957e-03, -1.17657084e+00, -1.79425733e-01,  1.31235585e-02,
       -1.93704399e+00,  3.02578499e-03, -2.87979161e-03,  4.19184656e+00,
       -4.27623166e-01,  8.74711390e-01,  3.00696818e-01])

Solver returns the same results (both RSS and the Beta_hat) for a very small initial values(zeros) and very large initial values(normal distribution with a large standard deviation)

### Trying Different Solver Choices

In [112]:
res_nelder_mead = minimize(RSS, x0= np.random.normal(0,1,X_train.shape[1]), args = (X_train, y_train), method ='Nelder-Mead', tol=10**-3)

In [113]:
res_nelder_mead.fun

597.2590466522504

In [114]:
res_nelder_mead.x

array([-9.67908759e-02, -3.05373987e-01,  1.41583363e+00, -7.59690681e-04,
       -2.88859014e+00,  8.10296926e-03, -5.19801264e-03,  5.82154072e+00,
       -6.94180784e-01,  3.29467575e-01,  2.76283651e-01])

In [115]:
res_powell = minimize(RSS, x0= np.random.normal(0,1,X_train.shape[1]), args = (X_train, y_train), method ='Powell', tol=10**-3)

In [116]:
res_powell.fun

545.535992678932

In [117]:
res_powell.x

array([ 8.46947289e-03, -1.17652126e+00, -1.79392974e-01,  1.31125119e-02,
       -1.93581672e+00,  3.02800165e-03, -2.88004170e-03,  4.19059717e+00,
       -4.27351615e-01,  8.74688502e-01,  3.00694951e-01])

In [118]:
###original solution
res.fun

545.5359871247364

In [119]:
res.x

array([ 8.44059572e-03, -1.17657071e+00, -1.79425465e-01,  1.31235832e-02,
       -1.93704486e+00,  3.02578789e-03, -2.87979537e-03,  4.19185798e+00,
       -4.27625892e-01,  8.74711237e-01,  3.00696782e-01])

Choice of the solver changes the results. In this case, both solvers were less performant than the original solver(one of BFGS, L-BFGS, SLSQP). However, Powell solver was much closer and the coefficients all have the same direction as the original solution.

### Regularization

\begin{equation}
argmin  \sum_{i=1}^{N} (y_{i} - B_{o} - \sum_{j=1}^{p} x_{ij}*B_{j})^2 + \lambda*\sum_{j=1}^{p}B_{j}^2
\end{equation}

In [120]:
def ridge(beta, X, y, lambda_):
    return ((y - lm_predict(X, beta))**2).sum() + (lambda_)*(beta**2).sum()

In [121]:
lambda_=0.01

In [122]:
res_ridge = minimize(ridge, x0= np.random.normal(0,1,X_train.shape[1]), args = (X_train, y_train, lambda_), tol = 10**-3)

In [123]:
res_ridge.x

array([ 9.41317098e-03, -1.17714066e+00, -1.80838178e-01,  1.31010560e-02,
       -1.91844273e+00,  3.00833664e-03, -2.86312367e-03,  4.13865533e+00,
       -4.15201547e-01,  8.73960656e-01,  3.00973351e-01])

In [124]:
res_ridge.fun

545.7711323717085

In [125]:
res.x

array([ 8.44059572e-03, -1.17657071e+00, -1.79425465e-01,  1.31235832e-02,
       -1.93704486e+00,  3.02578789e-03, -2.87979537e-03,  4.19185798e+00,
       -4.27625892e-01,  8.74711237e-01,  3.00696782e-01])

In [76]:
res_ridge.message

'Optimization terminated successfully.'

In [205]:
RSS(res_ridge.x, X_test,y_test)

122.46057009754884

There is very little change in the results, RSS and coefficients are still very similar. Also RSS in the test results are similar

In [222]:
output = dict()
for lambda_ in [0,.5,1,5,10]:
    res_ridge = minimize(ridge, x0= np.random.normal(0,1,X_train.shape[1]), args = (X_train, y_train, lambda_), tol=10**-3)
    output[lambda_] = [res_ridge.x, res_ridge.fun,RSS(res_ridge.x, X_test,y_test) ]    

In [223]:
##Test RSS
output[0][2], output[.5][2], output[1][2], output[5][2], output[10][2]

(122.4262801337932,
 123.99003656137126,
 124.9638166237936,
 126.7858957789673,
 127.1108642626774)

In [224]:
output[10][0]

array([ 0.07196888, -0.91380457, -0.07295217,  0.00661469, -0.22748663,
        0.00307556, -0.00198147,  0.38902101,  0.38230478,  0.64411446,
        0.3377126 ])

In [226]:
res.x

array([ 8.44068992e-03, -1.17657061e+00, -1.79425332e-01,  1.31235580e-02,
       -1.93704441e+00,  3.02578768e-03, -2.87979400e-03,  4.19185249e+00,
       -4.27624549e-01,  8.74711352e-01,  3.00696785e-01])

Output suggests that no L2 regularization minimizes the RSS in the test data. We can additionally observe that with high penalty, L2 at 10, We see much much smaller coefficients relative to the original

In [230]:
def lasso(beta, X, y, lambda_):
    return ((y - lm_predict(X, beta))**2).sum() + (lambda_)*(np.abs(beta)).sum()

In [240]:
output = dict()
for lambda_ in [0,.5,1,5,10,1000]:
    res_lasso = minimize(lasso, x0= np.random.normal(0,1,X_train.shape[1]), args = (X_train, y_train, lambda_))
    output[lambda_] = [res_lasso.x, res_lasso.fun,RSS(res_lasso.x, X_test,y_test) ]    

C:\Users\James\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1013: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (numpy.dot(yk, sk))


In [244]:
##Test RSS
output[0][2], output[.5][2], output[1][2], output[5][2], output[10][2], output[1000][2]

(122.42627139599216,
 122.68427642359296,
 123.01754973961307,
 125.36196061061278,
 128.59292679448922,
 140.26041693857604)

In [254]:
output[1000][0]

array([ 8.37079958e-02, -4.60123766e-09, -1.36890025e-09, -3.65388234e-09,
       -6.92654235e-09,  4.24027443e-03, -2.62766091e-09, -1.19732848e-08,
        2.54860025e-09, -2.76771624e-09,  4.64152887e-01])

Output suggests that no L1 regularization minimizes the RSS in the test data. We can additionally observe that with high penalty, L1 at 1000, We see some coefficients go to almost zero

Magnitude can absolute affect the regularization. For example, much smaller scaled feature leads to higher valued coefficient. When we do an L2 regularization on such features, they will be penalized higher because of the larger magnitude of the coefficient. This is in a sense unfair to the smaller scale features and could lead to less effective models